In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import copy

In [ ]:
# loading data
df_voice = pd.read_csv("./src/gender_voice_dataset.csv")

# Feature engineering


## Missing values

In [ ]:
df_voice.isna().sum()

Nie mamy żadnych brakujących wartości więc nie musimy się przejmować wypełnianiem braków.

## Outliers

---

In [ ]:
df_voice.shape

In [ ]:
columns = df_voice.columns[1:20]   
for column in columns:
    factor = 4
    upper_lim = df_voice[column].mean () + df_voice[column].std () * factor
    lower_lim = df_voice[column].mean () - df_voice[column].std () * factor
    df_voice = df_voice[(df_voice[column] < upper_lim) & (df_voice[column] > lower_lim)]
df_voice.reset_index(inplace = True)
df_voice.drop(axis = 1, columns = ['index'], inplace = True)

In [ ]:
df_voice.shape

Aby pozbyć się wartości odstających postanowiliśmy usunąć te wartości, które znajdują się w dowolnej kolumnie dalej od średniej niż 4 odchylenia standardowe. Straciliśmy w ten sposób około 200 obserwacji (~ 7 %)

# Usuwamy mocno skorelowane kolumny

---

In [ ]:
c = df_voice.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")
so = so.loc[(so < 1) & (so > 0.8)]
so[::-1]

Bardzo dużo kolumn jest skorelowanych, więc aby uprościć nasz model usuwamy kolumny, których współczynnik skorelowania jest wyższy niż 0.8 . W przypadku decyzji, którą z kolumn usunąć, usuwaliśmy tą która była w mniejszym stopniu skorelowana z targetem.

In [ ]:
df_voice.drop(axis = 1, columns=['maxdom','meanfreq','centroid','skew',"sfm","sp.ent","sd"],inplace = True)

In [ ]:
df_voice.shape

In [ ]:
df_voice.describe()

In [ ]:
df_voice.hist(bins = 40, figsize=(18, 12))
plt.show()

# Standaryzacja

Standaryzujemy dane aby były zcentrowane do zera i żeby ich wariancje były tego samego rzędu. Robimy to po to, aby żadna zmienna nie dominowała i model mógł się uczyć na wszystkich zmiennych.

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df_voice.drop("label",axis=1))
df_voice_standard = pd.DataFrame(scaler.transform(df_voice.drop("label",axis=1)),columns=['median','Q25','Q75','IQR','kurt','mode','meanfun','minfun','maxfun','meandom','mindom','dfrange','modindx'])
df_voice_standard.describe()

In [ ]:
df_voice_standard.hist(bins = 40, figsize=(18, 12))
plt.show()

# Dyskretyzacja

Dyskretyzacji używamy dla nieregularnych rozkładów *mode i dfrange* aby nasze modele lepiej sobie z nimi radziły.

In [ ]:
disc = preprocessing.KBinsDiscretizer(n_bins=20, encode='ordinal',strategy="uniform").fit(df_voice_standard[['mode','dfrange']])
df_voice_standard[['mode','dfrange']] = disc.transform(df_voice_standard[['mode','dfrange']])

In [ ]:
df_voice_standard.hist(bins = 40, figsize=(18, 12))
plt.show()

# Baseline model

Pierwszy model baselinowy to najprostsza instrukcja warunkowa, na kolumnie "meanfun", w której wartości bardzo ładnie separują się w zależności od targetu. Jak zobaczymy już tak banalny model daje nam trafność na poziomie 95 %.

In [ ]:
df_voice_disc['label'] = df_voice[['label']]
df_voice_standard['label'] = df_voice[['label']]

In [ ]:
def baseline_model(data):
    ## warunek meanfun < 0.13 -> male, otherwise: female
    suma = 0
    for i in range(len(data)):
        if(data["meanfun"][i] < 0.14):
            predicted = 'male'
        else:
            predicted = 'female'
        if(predicted == data['label'][i]):
            suma = suma + 1
    return suma / len(data)

In [ ]:
baseline_model(df_voice)

# Baseline model x2
Drugi baselinowym modelem będzie regresja logistyczna bez regularyzacji. Trafność modelu to 98 %.

In [ ]:
# ze standaryzacją 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
df_voice_standard['label'] = df_voice[['label']]
X = df_voice_standard.drop('label',axis=1)
y=df_voice_standard[['label']]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)
lr_base = LogisticRegression(max_iter=1000,penalty="none").fit(X_train,y_train)
y_pred = lr_base.predict(X_test)
accuracy_score(y_test,y_pred)